In [1]:
import openai
import json
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def get_current_weather(location, unit = "fahrenheit"):
    """
    Retreive the current temperature for a temperature
    """
    weather_info = {
        'location': location,
        'temperature': 70,
        'unit': unit,
        'forecast': ['sunny', 'windy']
    }
    return json.dumps(weather_info)


def get_member_count(date = 20230630, population = 'all'):
    """
    Retreive the current number of members for a given date and population.

    population can one of the following values: 'all', '1', '2', or '3'
    """
    member_count = {
        'date': date,
        'population': population
    }
    return member_count

In [14]:
completion = openai.ChatCompletion.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": "How many members were there in July 2021?"}],
    functions=[
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    },
    {
        "name": "get_member_count",
        "description": "Retreive the current number of members for a given date and population.",
        "parameters": {
            "type": "object",
            "properties": {
                "date": {
                    "type": "integer",
                    "description": "The Y/m/d in the query. e.g. 20200131, or 20220630",
                },
                "population": {
                    "type": "string",
                    "description": "Group of people to count. Can be one of the following values: 'all', '1', '2', or '3'"
                },
            },
            "required": ["date", "population"],
        },
    },
],
function_call="auto",
)

In [15]:
reply_content = completion.choices[0]
reply_content

<OpenAIObject at 0x7f843715f1d0> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "get_member_count",
      "arguments": "{\n  \"date\": 20210731,\n  \"population\": \"all\"\n}"
    }
  },
  "finish_reason": "function_call"
}

In [16]:
reply_content.to_dict()

{'index': 0,
 'message': <OpenAIObject at 0x7f843715fe30> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_member_count",
     "arguments": "{\n  \"date\": 20210731,\n  \"population\": \"all\"\n}"
   }
 },
 'finish_reason': 'function_call'}

In [17]:
function_name = reply_content.to_dict()['message']['function_call']['name']
args = json.loads(reply_content.to_dict()['message']['function_call']['arguments'])

In [18]:
function_name

'get_member_count'

In [19]:
args

{'date': 20210731, 'population': 'all'}

In [22]:
# using repr to add quotes
output = f"{function_name}({', '.join(repr(args[key]) for key in args.keys())})"
output

"get_member_count(20210731, 'all')"

In [23]:
eval(output)

{'year': 20210731, 'population': 'all'}